In [1]:
from ast import literal_eval
from datetime import datetime
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


sns.set_theme(style='white',font_scale=1.5)
sns.set_theme(rc={'axes.facecolor':'#FFFAF0','figure.facecolor':'#FFFAF0'})
sns.set_context('poster',font_scale = .7)

import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 42

DATA_DIR = Path().resolve().parent / 'data'

In [2]:
def check_df(dataframe, head=5):
    print('SHAPE'.center(70,'-'))
    print(dataframe.shape)
    print('HEAD'.center(70,'-'))
    print(dataframe.head(head))
    print('INFO'.center(70,'-'))
    print(dataframe.info())
    print('NUNIQUE'.center(70,'-'))
    print(dataframe.nunique())
    print('MISSING VALUES'.center(70,'-'))
    print(dataframe.isnull().sum())
    print('DUPLICATED ROWS'.center(70,'-'))
    print(dataframe.duplicated().sum())

## Загрузка данных

In [3]:
df = pd.read_csv(DATA_DIR / 'Spb_flats_prices.csv')
check_df(df)

--------------------------------SHAPE---------------------------------
(1080, 9)
---------------------------------HEAD---------------------------------
                                   Underground Times  \
0  {'Underground name': ['Балтийская', 'Василеост...   
1  {'Underground name': ['Московская', 'Бухарестс...   
2  {'Underground name': ['Василеостровская', 'Спо...   
3  {'Underground name': ['Улица Дыбенко', 'Проспе...   
4  {'Underground name': ['Девяткино', 'Граждански...   

   Flat Status (1 - handed over, 0 - not handed over) Flat Type  \
0                                                  1     4-комн   
1                                                  1     2-комн   
2                                                  0     3-комн   
3                                                  1     2-комн   
4                                                  1     1-комн   

   Total Area (m^2)  Living Area (m^2)  Kitchen Area (m^2)  Floor  \
0            125.50               75.8 

In [4]:
df.columns = ['metro_info',
             'flat_status',
             'num_of_rooms',
             'total_area_m2',
             'living_area_m2',
             'kitchen_area_m2',
             'floor',
             'year_of_bulding',
             'price']

In [5]:
def nearest_metro_station(json):
    stations_dict = literal_eval(json)

    nearest_station = stations_dict['Underground name'][0]
    time_to_get = stations_dict['Underground time'][0]
    way_to_get_to = stations_dict['Transport type'][0]

    return [nearest_station, time_to_get, way_to_get_to]

In [6]:
df[['metro_station',
    'minutes_to_metro',
    'transfer_type']] = pd.DataFrame(df['metro_info'].apply(lambda x: nearest_metro_station(x)).tolist(),
                                     index=df.index)
df.drop('metro_info', axis=1, inplace=True)

In [7]:
df['num_of_rooms'].unique()

array(['4-комн', '2-комн', '3-комн', '1-комн', '5-комн', 'Студия',
       'Многокомнатны', 'Многокомнатна', 'Апартаменты-студия', '15/4',
       'Квартир', '29/4'], dtype=object)

In [8]:
df['num_of_rooms'] = df['num_of_rooms'].apply(lambda x: 'Неизвестно' if x in ['Многокомнатны',
                                                                          'Многокомнатна',
                                                                          '15/4',
                                                                          'Квартир',
                                                                          '29/4'] else x)

In [9]:
replacements = {
    'Апартаменты-студия': 'Студия',
    '1-комн': '1-комнатная'
}
df['num_of_rooms'] = df['num_of_rooms'].replace('Апартаменты-студия', 'Студия')

In [10]:
df['num_of_rooms'] = df['num_of_rooms'].apply(lambda x: x + 'атная' if x not in ['Студия', 'Неизвестно'] else x)

In [11]:
df['num_of_rooms'].unique()

array(['4-комнатная', '2-комнатная', '3-комнатная', '1-комнатная',
       '5-комнатная', 'Студия', 'Неизвестно'], dtype=object)

In [12]:
df.duplicated().sum()

np.int64(82)

In [13]:
current_year = datetime.now().year
df['house_age'] = (current_year - df['year_of_bulding']).apply(lambda x: x if x > 0 else 0)

In [14]:
df['is_future_building'] = (df['year_of_bulding'] > current_year).astype(int)

In [15]:
df.head()

,flat_status,num_of_rooms,total_area_m2,living_area_m2,kitchen_area_m2,floor,year_of_bulding,price,metro_station,minutes_to_metro,transfer_type,house_age,is_future_building
0,1,4-комнатная,125.50,75.8,22.9,4,1876,24500000,Балтийская,5,car,149,0
1,1,2-комнатная,41.50,26.4,6.0,3,1964,6300000,Московская,6,car,61,0
2,0,3-комнатная,80.39,44.9,10.2,3,2025,30061520,Василеостровская,11,legs,0,0
3,1,2-комнатная,65.10,27.0,19.0,4,1983,11500000,Улица Дыбенко,7,car,42,0
4,1,1-комнатная,38.50,16.6,10.5,10,2015,5599000,Девяткино,6,car,10,0


In [16]:
df['minutes_to_metro'] = df['minutes_to_metro'].replace('откроется', 10).astype(int)

In [17]:
check_df(df)

--------------------------------SHAPE---------------------------------
(1080, 13)
---------------------------------HEAD---------------------------------
   flat_status num_of_rooms  total_area_m2  living_area_m2  kitchen_area_m2  \
0            1  4-комнатная         125.50            75.8             22.9   
1            1  2-комнатная          41.50            26.4              6.0   
2            0  3-комнатная          80.39            44.9             10.2   
3            1  2-комнатная          65.10            27.0             19.0   
4            1  1-комнатная          38.50            16.6             10.5   

   floor  year_of_bulding     price     metro_station  minutes_to_metro  \
0      4             1876  24500000        Балтийская                 5   
1      3             1964   6300000        Московская                 6   
2      3             2025  30061520  Василеостровская                11   
3      4             1983  11500000     Улица Дыбенко                 7 

In [18]:
df.drop_duplicates(inplace=True)

# EDA